# Plot templates (WIP)

In [3]:
import sys
import os
import pandas as pd
import numpy as np
import time
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta


# For creating cool charts :)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.ticker as mtick
from matplotlib import image
import matplotlib as mpl
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.dates as mdates

# We load pre-defined functions, fonts and stylesheet.
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\utils")
    sys.path.append(module_path+"\\assets")

from tukan_helper_functions import *

path = [x for x in sys.path if "utils" in x][0]

# Optional for TUKAN's mplstylesheet
plt.style.use(module_path + '\\utils\\tukan_style.mpl')

### Payload

In [4]:
# Main payload
payload = {
    "type": "data_table",
    "operation": "sum",
    "language": "es",
    "group_by": [
        "economic_activity"
    ],
    "categories": {
        "economic_activity": [
            "6b36ca46b6cfd91",
            "457155464609a2f",
            "36348912d8470dd",
            "faa2a8d0af8a72c",
            "be676b5dd921cb7",
            "f5adaadda584ca7",
            "e426cc87d0540ab",
            "afcc312ccddfcc1",
            "990b94ebe38c9ca",
            "d35f5b82779e7d5",
            "3726993cc9fecab",
            "bbb49ae78601ab9",
            "4bc9836c2d7e60a",
            "fcb303b72a98f6c",
            "d05c3b2b73d75fc",
            "44d246411040129",
            "a07267f78158c2c",
            "feb7bb4445c808d",
            "169c33ccdd66d77",
            "23cf92d98dd7c11"
        ]
    },
    "request": [
        {
            "table": "mex_inegi_gdp_national",
            "variables": [
                "675882bced9ce54"
            ]
        }
    ],
    "from": "2011-01-01"
}

response = get_tukan_api_request(payload)

Success getting the data


### Data Transformations

In [21]:
# Data
gdp = response["data"]

# Unit in millions
gdp['675882bced9ce54'] = gdp['675882bced9ce54']/1000000

# Remove the quarters from previous years that are not available yet for the current year
# date_filter = gdp['date'].max().month
# gdp = gdp[gdp["date"].dt.month<= date_filter]
# # Calculate the cumulative GDP for each year and economic activity, dividing it by a million 
# gdp = gdp.groupby([gdp.date.dt.month,'economic_activity__ref','economic_activity']).sum().reset_index()


# gdp[(gdp['date'] > '2013-01-01') & (gdp['date'] < '2013-02-01')]
# max_year=gdp['date'].max().year
# max_mon=gdp['date'].max().month
# last_year_1 = pd.to_datetime

current_qtr = gdp['date'].tail(1)
last_qtr = current_qtr - pd.DateOffset(months=3)
lastyear_qtr = current_qtr - pd.DateOffset(years=1)
# lastyear_qtr.dt.strftime('%Y-%m-%d')

# temp = gdp[(gdp['Date'].dt.date.astype(str) == lastyear_qtr.dt.date.astype(str)) | (gdp['Date'].dt.date.astype(str) == current_qtr.dt.date.astype(str))] #| (gdp['Date'].dt.date.astype(str) == '2017-03-20')]
# temp

# lastyear_qtr.to_string()


# gdp

KeyError: 'Date'

In [6]:


# Pivot the data frame so it is easy to plot, calculate the variation between 2021-2019 and sort ascendingly according to it
gdp = gdp.pivot(index = 'economic_activity', columns='date', values='675882bced9ce54').reset_index()
gdp['var_2021_2019'] =  (gdp[2021] / gdp[2019])-1
gdp = gdp.sort_values('var_2021_2019', ascending=True)
gdp['p_ref'] =  range(gdp.shape[0]) #plot_reference

# Rename the tickers to improve plot design
gdp = gdp.replace({'Servicios de esparcimiento culturales y deportivos, y otros servicios recreativos':'Serv. de esparcimiento y recreativos',
                   'Generación, transmisión, distribución y comercialización de energía eléctrica, suministro de agua y de gas natural por ductos al consumidor final':'Energía, suministro de agua y gas natural',
                   'Servicios de alojamiento temporal y de preparación de alimentos y bebidas':'Hoteles y Restaurantes',
                   'Otros servicios excepto actividades gubernamentales':'Otros servicios',
                   'Agricultura, cría y explotación de animales, aprovechamiento forestal, pesca y caza':'Agricultura, ganadería, pesca y acts. forestales',
                   'Servicios de apoyo a los negocios y manejo de residuos, y servicios de remediación':'Serv. de apoyo a los negocios y manejo de residuos',
                   'Actividades legislativas, gubernamentales, de impartición de justicia y de organismos internacionales y extraterritoriales':'Acts. gubernamentales y legislativas',
                   'Servicios inmobiliarios y de alquiler de bienes muebles e intangibles':'Serv. inmobiliarios y alquiler de bienes',
                   'Servicios de salud y de asistencia social':'Serv. sociales y de salud',
                   'Servicios profesionales, científicos y técnicos':'Serv. profesionales, científicos y técnicos',
                   'Servicios educativos':'Serv. educativos',
                   'Servicios financieros y de seguros':
                   'Serv. financieros y de seguros'
 })
gdp.reset_index(drop=True, inplace=True)
gdp

date,economic_activity,2016,2017,2018,2019,2020,2021,var_2021_2019,p_ref
0,Hoteles y Restaurantes,1147265.607,1201528.328,1228423.709,1241861.893,676994.783,870110.622,-0.299350,0
1,Serv. de esparcimiento y recreativos,226592.249,229521.481,237896.280,226903.238,130205.131,169301.545,-0.253860,1
2,Otros servicios,1075234.867,1072203.236,1078365.397,1106508.274,927385.879,938125.115,-0.152175,2
3,Construcción,3813327.696,3798457.272,3879960.768,3711075.695,2974242.021,3248462.180,-0.124658,3
4,"Transportes, correos y almacenamiento",3340797.198,3465620.065,3592152.825,3602808.963,2802816.552,3224937.897,-0.104882,4
5,Serv. de apoyo a los negocios y manejo de resi...,1721361.234,1846638.400,1949577.416,2042615.645,2044393.754,1863279.234,-0.087797,5
6,Serv. financieros y de seguros,2338827.998,2493403.012,2596446.266,2707990.094,2574662.266,2476279.257,-0.085566,6
7,"Energía, suministro de agua y gas natural",808562.261,806273.255,870359.091,857280.272,809453.418,807583.840,-0.057970,7
8,Serv. educativos,2010293.147,2029705.660,2040122.727,2044636.818,2013317.543,1982590.430,-0.030346,8
9,Industrias manufactureras,8387019.523,8647191.571,8817634.943,8888248.249,7780052.534,8629602.866,-0.029100,9
